# Imputation using a Mixture of Gaussians and the EM algorithm

In [ ]:
import numpy as np
np.random.seed(42)
import pandas as pd
from sklearn.datasets import load_boston
from scipy import stats
from scipy import linalg
from sklearn.cluster import KMeans

In [ ]:
X = np.genfromtxt("../data/boston-50-MCAR.csv", delimiter=",")

In [ ]:
pd.DataFrame(X)

In [ ]:
masked_X = X.copy()

In [ ]:
mean = np.nanmean(masked_X, axis=0)
std = np.nanstd(masked_X, axis=0)
scaled_X = (masked_X - mean)/std

## now create a bunch of gaussians for the mixture model

In [ ]:
num_gaussians = 3
num_features = X.shape[1]
num_examples = X.shape[0]
# μs = np.random.rand(num_gaussians, num_features)
# Σs = np.stack([np.identity(num_features) for _ in range(num_gaussians)], axis=0)
indices = np.stack([np.random.choice(num_examples, int(num_examples/2)) for _ in range(num_gaussians)], axis = 0)
μs = np.stack([np.nanmean(X[idx, :], axis=0) for idx in indices], axis=0)
Σs = np.stack(
    [np.nanmean(
        [np.outer(X[i,:] - μ, X[i,:] - μ) for i in idx], axis=0) for μ, idx in zip(μs, indices)]
    , axis=0)

In [ ]:
# # use k-means to come up with the initial means and varainces
# μs = np.zeros(shape=(num_gaussians, num_features))
# Σs = np.zeros(shape=(num_gaussians, num_features, num_features))
# mean_imputed_X = scaled_X
# mean_imputed_X[np.isnan(mean_imputed_X)] = np.nanmean(scaled_X, axis=0)[np.where(np.isnan(mean_imputed_X))[1]]
# mean_imputed_X += np.random.random(mean_imputed_X.shape)*0.01
# kmeans = KMeans(n_clusters=num_gaussians, random_state=0).fit(mean_imputed_X)
# for j in range(num_gaussians):
#     locs = np.where(kmeans.labels_ == j)[0]
#     μs[j,:] = np.mean(mean_imputed_X[locs,:], axis=0)
#     diff = mean_imputed_X[locs,:] - μs[j,:]
#     Σs[j,:,:] = (diff.T @ diff)/diff.shape[0] 

In [ ]:
def gmm_ll(imputed_X, μs, Σs, ps):
    ll = 0
    for i in range(imputed_X.shape[0]):
        tmp = 0
        for j in range(μs.shape[0]):
            tmp += ps[i,j] * stats.multivariate_normal.pdf(imputed_X[i,:], mean=μs[j,:], cov=Σs[j,:,:], allow_singular=True)
        ll += np.log(tmp)
    return ll

In [ ]:
def gmm_impute(scaled_X, μs, Σs, ps):
    Xs = np.stack([scaled_X]*num_gaussians, axis=0)

    for i in range(X.shape[0]):
        x_row = scaled_X[i,:]

        if np.all(~np.isnan(x_row)): continue

        o_locs = np.where(~np.isnan(x_row))[0]
        m_locs = np.where(np.isnan(x_row))[0]
        oo_coords = tuple(zip(*[(i, j) for i in o_locs for j in o_locs]))
        mo_coords = tuple(zip(*[(i, j) for i in m_locs for j in o_locs]))

        for j in range(μs.shape[0]):
            diff = x_row[o_locs] - μs[j,o_locs]
            
            Xs[j,i,m_locs] = μs[j,m_locs]
            
            if (len(o_locs)):
                Σoo = Σs[j, :, :][oo_coords].reshape(len(o_locs),len(o_locs))
                Σmo = Σs[j, :, :][mo_coords].reshape(len(m_locs),len(o_locs))

                Xs[j,i,m_locs] += Σmo @ linalg.inv(Σoo) @ diff
            
    imputed_X = np.zeros_like(scaled_X)
    for j in range(μs.shape[0]):
        for i in range(X.shape[0]):
            imputed_X[i,:] += ps[i,j]*Xs[j,i,:]
            
    return Xs, imputed_X

In [ ]:
prev_LL = -np.inf
for _ in range(100):
    # E-step
    qs = np.zeros(shape=(num_examples,num_gaussians))
    for i in range(num_examples):
        x_row = scaled_X[i,:]
        o_locs = np.where(~np.isnan(x_row))[0]
        oo_coords = tuple(zip(*[(i, j) for i in o_locs for j in o_locs]))

        x = x_row[o_locs]
        sz = len(x)

        for j in range(num_gaussians):
            if (len(o_locs)):
                Σoo = Σs[j, :, :][oo_coords].reshape(sz,sz)
                μo = μs[j, o_locs]

#             qs[i,j] = 1/np.sqrt(np.power(2*np.pi,sz)*linalg.det(Σoo))\
#                     *np.exp(-0.5 * (x - μo).T @ linalg.inv(Σoo) @ (x - μo))
                qs[i,j] = stats.multivariate_normal.pdf(x, mean=μo, cov=Σoo, allow_singular=True)
            else:
                qs[i,j] = np.random.rand(1)

    ps = qs/np.sum(qs, axis=1, keepdims=True)

    # M-step
    # first fill in the missing values with each gaussian
    Xs, _ = gmm_impute(scaled_X, μs, Σs, ps)  
    
    # save the current estimates for the params incase the LL gets worse
    μs_best, Σs_best = μs.copy(), Σs.copy()
    
    # now recompute μs
    for j in range(num_gaussians):
        p = ps[:,j]
        μs[j] = (p @ Xs[j])/np.sum(p)

    # and now Σs
    for j in range(num_gaussians):

        p = ps[:,j]

        # calc C
        C = np.zeros(shape=(num_features, num_features))
        for i in range(num_examples):
            x_row = scaled_X[i,:]

            if np.all(~np.isnan(x_row)): continue

            o_locs = np.where(~np.isnan(x_row))[0]
            m_locs = np.where(np.isnan(x_row))[0]
            oo_coords = tuple(zip(*[(i, j) for i in o_locs for j in o_locs]))
            mo_coords = tuple(zip(*[(i, j) for i in m_locs for j in o_locs]))
            mm_coords = tuple(zip(*[(i, j) for i in m_locs for j in m_locs]))

            Σmm = Σs[j, :, :][mm_coords].reshape(len(m_locs),len(m_locs))

            tmp = Σmm
            if (len(o_locs)):
                Σoo = Σs[j, :, :][oo_coords].reshape(len(o_locs),len(o_locs))
                Σmo = Σs[j, :, :][mo_coords].reshape(len(m_locs),len(o_locs))
                tmp -= Σmo @ linalg.inv(Σoo) @ Σmo.T
                
            tmp = p[i]/np.sum(p)*tmp
            C[mm_coords] += tmp.reshape(len(m_locs)**2)
            

        Σs[j] = np.zeros_like(C)
        for i in range(num_examples):
            diff = Xs[j,i,:] - μs[j]
            Σs[j] += np.outer(diff, diff.T)*p[i]

        Σs[j] /= np.sum(p)
        Σs[j] += C
        # regularisation term ensuring that the cov matrix is always pos def
        Σs[j] += np.diag(np.ones(shape=(num_features,))*1e-3)
       
    _, imputed_X = gmm_impute(scaled_X, μs, Σs, ps)
    LL = gmm_ll(imputed_X, μs, Σs, ps)
    if (LL < prev_LL or LL - prev_LL < 1e-3):
        μs, Σs = μs_best, Σs_best
        break
        
#     print("RMSE: %s" % np.sqrt(np.mean(np.power(imputed_X*std + mean - X,2))))
    print("LL: %s" % LL)
    
    prev_LL = LL

In [ ]:
(np.array([0.1,0.2,0.5]) @ np.array([[1,2,3],[4,5,6],[7,8,9]]))/np.sum([0.1,0.2,0.5])

In [ ]:
np.sum(np.array([[0.1, 0.2, 0.5][i] * np.array([[1],[4],[7]])[i,:] for i in range(3)]), axis=0)/np.sum([0.1,0.2,0.5])

In [ ]:
display(qs)

In [ ]:
tq = np.array([[1,2,3],[4,5,6],[7,8,9],[10,11,12],[0,0,0]])
display(tq)

In [ ]:
tq/np.sum(tq, axis=1, keepdims=True)

In [ ]:
np.array([[tq[i,j]/np.sum(tq[i,:]) for j in range(3)] for i in range(5)])

In [ ]:
np.random.random(4)